In [277]:
# import libraries

import json
import pandas as pd
import random

from utils import load_filtered_data
from surprise.model_selection import train_test_split, cross_validate
from surprise import Dataset, Reader, NormalPredictor, KNNBasic, KNNWithZScore, KNNWithMeans, KNNWithZScore, SVD, accuracy

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import NearestNeighbors

city_info = load_filtered_data('Sicklerville')

In [278]:
with open('communities/Sicklerville_business_reviews_communities.json') as f:
    data = json.load(f)
    
print(data['communities'])

[['RgDVC3ZUBqpEe6Y1kPhIpw', 'FV6QtuA5P52Cm5q7SOeMLQ', 'JSpwp4pAp_Hu4xmPuNitBA', 'UKjsdQvypama0X_henp77A', 'p_qFRYEqyT93hgzCJ4Jnxw', '1C5kFjSPdf9P74IhPy6e-w', '3Qqi2SFrqZDmzb_txv3yaQ', 'iJe_4Z_731GUnEFYZ8xREw', 'XDHIa7cmHBL47XYcPPe5Rw', 'hof0egGfooFnQOSPscC7vA', 'YV7cc6mX_58JzMbj13TF5A', '1-icdXJv2v9MMbytcVLBcQ', 'e0vKaw6oJl0o-aou8YzVNw', '8DGc1UEegk6SBGV39OJtQA', 'ET8n-r7glWYqZhuR6GcdNw', 'ypAXUlbv4GCbI-l2O1JfXw', 'W8HUz05IY7AkX70BaSSwFw', '4TE2IPdI9_4dYAalTGImUg', 'uoeB_8Cff5zFWWzzXUOLNw', 'NRBvM9AHOPYSXMB45RkZ9g', '1n3IENFR9dpPztzTh0A6dA', '0Q3dKYtiXehjNlTjiMD8dg', 'uxkOQHfcphFghmwUYjEvng', 'M6fECH_et0ffSODBrvQABA'], ['zkamNMEjihh3zN7lC7_WVw', 'LI_A-_R6SQotk6fX8C0nPQ', 'ApJ9YgYU-AhS4a-F5oTGHw', 'A_BF2dDDUTKGVXrqxO9mag', 'B-s-8VUnuBjGTP3d01jsyw', 'WTNE5YK9RTgP5jzStlbWjQ', 'wpfvF8ue3yr1m5xD4OrZNw', 'ZTvil7E2JbtORKm0bu6MKQ', 'Vm0fyrUjwMTDgwqV6DhqDw', 'KwC7I2KR-HlEb3KSH4a0bw', 'uIKD7cJF4ZWkicin2ulWPg', 'Djxt8nbOFGJK6F0bzOnL3g', '0ZUHtvIiZ1DPKKDTiieuKA', 'HiKXqt366G3FmHZyIxM20g', '8ybqiII

In [279]:
# Define evaluation function
def evaluate_algorithm(algo, trainset, testset):
    algo.fit(trainset)
    predictions = algo.test(testset)
    
    # Compute and return RMSE
    rmse = accuracy.rmse(predictions)
    return rmse

In [280]:
merged = pd.merge(city_info['review'], city_info['user'], on='user_id', how='left')
merged = pd.merge(merged, city_info['business'], on='business_id', how='left')
merged = merged.drop_duplicates(subset=['user_id', 'business_id'])


In [281]:
# Recommend top N items for a user using a recommender model
def recommend_top_n(algo, trainset, user_id, n=10):
    user_ratings = trainset.ur[user_id]
    items = [item_id for (item_id, _) in user_ratings]
    
    item_scores = {}
    for item_id in trainset.all_items():
        if item_id not in items:
            prediction = algo.predict(trainset.to_raw_uid(user_id), trainset.to_raw_iid(item_id), verbose=True)
            item_scores[item_id] = prediction.est
    
    top_items = sorted(item_scores, key=item_scores.get, reverse=True)[:n]

    #from raw_id to actual_id
    return [trainset.to_raw_iid(i) for i in top_items]

def get_users(df_testset, trainset):
    # we just want to recommend positive ratings
    pos_rating = 3
    df_testset_pos = df_testset[df_testset["stars_x"] > pos_rating]
    # which users exist in the training and testset
    users = []
    for u in df_testset_pos["user_id"].unique():
        try :
            trainset.to_inner_uid(u)
            users.append(u)
        except ValueError:
            continue
    return users

In [282]:
def get_community_recommendations(community, merged, min_ratings_for_split=5, test_size=1):
    global count
    community_reviews = merged[merged['user_id'].isin(community)]
    if len(community_reviews) < min_ratings_for_split:
        count = count + 1
        return None
    
    reader = Reader(rating_scale=(1, 5))  # Ratings scale is from 1 to 5
    surprise_dataset = Dataset.load_from_df(community_reviews[['user_id', 'business_id', 'stars_x']], reader)
    
    
    trainset, testset = train_test_split(surprise_dataset, test_size=test_size)

    algo = KNNBasic(sim_options={'user_based': True})
    algo.fit(trainset)
    predictions = algo.test(testset)
    rmse = accuracy.rmse(predictions)
    
    return rmse

# Generate recommendations for each community
community_recommendations = {}
count = 0
id = 0
for community in data['communities']:
    recommendation = get_community_recommendations(community, merged)
    if recommendation is not None:
        community_recommendations[id] = recommendation
    id = id + 1
    



TypeError: Singleton array array(<surprise.dataset.DatasetAutoFolds object at 0x705c9b570670>,
      dtype=object) cannot be considered a valid collection.